### Set up

#### 1. Set  up  accounts and role

In [1]:
#!pip install sagemaker==1.39.0

In [2]:
import sagemaker
import boto3

sagemaker_session = sagemaker.Session()
account_id =  boto3.client('sts').get_caller_identity().get('Account')
region = boto3.session.Session().region_name


#role = sagemaker.get_execution_role()
role="arn:aws:iam::{}:role/service-role/AmazonSageMaker-ExecutionRole-20190118T115449".format(account_id)


#### 2. Setup image and instance type

In [3]:
pytorch_custom_image_name="ppi-extractor:gpu-1.0.0-201910130520"
instance_type = "ml.p3.8xlarge" 

In [4]:
docker_repo = "{}.dkr.ecr.{}.amazonaws.com/{}".format(account_id, region, pytorch_custom_image_name)

#### 3. Configure train/ test and validation datasets

In [5]:
bucket = "aegovan-data"

In [6]:
trainfile = "s3://{}/processed_dataset/train_unique_pub_v6_less_negative.json".format(bucket)
testfile= "s3://{}/processed_dataset/test_unique_pub_v6_less_negative.json".format(bucket)
valfile="s3://{}/processed_dataset/val_unique_pub_v6_less_negative.json".format(bucket)


trainfile = "s3://{}/processed_dataset/train_unique_pub_v6_max_neg.json".format(bucket)
testfile= "s3://{}/processed_dataset/test_unique_pub_v6_max_neg.json".format(bucket)
valfile="s3://{}/processed_dataset/val_unique_pub_v6_max_neg.json".format(bucket)
# trainfile = "s3://{}/aimed/train_unique_negative_entity_only.json".format(bucket)
# testfile= "s3://{}/aimed/test_unique_negative_entity_only.json".format(bucket)
# valfile= "s3://{}/aimed/val_unique_negative_entity_only.json".format(bucket)



pretrained_bert="s3://{}/embeddings/bert/".format(bucket)


s3_output_path= "s3://{}/results/".format(bucket)
s3_code_path= "s3://{}/aimed_bert_code".format(bucket)

### Start training

In [7]:
pub_inputs = {
    "train" : trainfile,
    "val" :valfile,
    "test" : testfile,
    "PRETRAINED_BIOBERT" : pretrained_bert
}

In [8]:
BertNetworkFactoryhyperparameters = {
    "dataset":"PpiDatasetFactory",
    "network" :"RelationExtractorBioBertFactory",
    "trainfile":trainfile.split("/")[-1],
     "valfile":valfile.split("/")[-1],
    "testfile":testfile.split("/")[-1],
    "batchsize": "8",
    "accumulation_steps": 8,
    "epochs" : "1000",   
    "log-level" : "INFO",
    "learningrate":.00001,
    
    "earlystoppingpatience":20
}

In [9]:
BertNoType_NetworkFactoryhyperparameters = {
    "dataset":"PpiNoInteractionDatasetFactory",
    "network" :"RelationExtractorBioBertFactory",
    "trainfile":trainfile.split("/")[-1],
     "valfile":valfile.split("/")[-1],
    "batchsize": "8",
    "epochs" : "1000",   
    "log-level" : "INFO",
    "learningrate":.00001,
    "earlystoppingpatience":20
}

In [10]:
metric_definitions = [{"Name": "TrainLoss",
                     "Regex": "###score: train_loss### (\d*[.]?\d*)"}
                    ,{"Name": "ValidationLoss",
                     "Regex": "###score: val_loss### (\d*[.]?\d*)"}
                    ,{"Name": "TrainFScore",
                     "Regex": "###score: train_fscore### (\d*[.]?\d*)"}
                   ,{"Name": "ValidationFScore",
                     "Regex": "###score: val_fscore### (\d*[.]?\d*)"}
                    ]

In [11]:
!git log -1 | head -1
!git log -1 | head -5 | tail -1

commit b88fc61617018334491247162e33042bf5c2eefc
    Fix predictor


In [12]:
git_config = {'repo': 'https://github.com/elangovana/PPI-typed-relation-extractor.git',
              'branch': 'master',
            #  'commit': '58a09e154935248667062a36fdae7d86b86b477c'
             }

In [13]:
hyperparameters =BertNetworkFactoryhyperparameters #BertNoType_NetworkFactoryhyperparameters 
inputs = pub_inputs 


In [14]:
hyperparameters

{'accumulation_steps': 8,
 'batchsize': '8',
 'dataset': 'PpiDatasetFactory',
 'earlystoppingpatience': 20,
 'epochs': '1000',
 'learningrate': 1e-05,
 'log-level': 'INFO',
 'network': 'RelationExtractorBioBertFactory',
 'testfile': 'test_unique_pub_v6_max_neg.json',
 'trainfile': 'train_unique_pub_v6_max_neg.json',
 'valfile': 'val_unique_pub_v6_max_neg.json'}

In [15]:
inputs

{'PRETRAINED_BIOBERT': 's3://aegovan-data/embeddings/bert/',
 'test': 's3://aegovan-data/processed_dataset/test_unique_pub_v6_max_neg.json',
 'train': 's3://aegovan-data/processed_dataset/train_unique_pub_v6_max_neg.json',
 'val': 's3://aegovan-data/processed_dataset/val_unique_pub_v6_max_neg.json'}

In [16]:
from sagemaker.pytorch import PyTorch

estimator = PyTorch(
     #entry_point='main_train_k_fold.py',
    entry_point='main_train_bert.py',
                    source_dir = 'source/algorithms',
                    dependencies =['source/algorithms', 'source/datasets', 'source/preprocessor', 'source/modelnetworks','source/trainpipelinesbuilders'],
                    role=role,
                    framework_version ="1.0.0",
                    py_version='py3',
                    git_config= git_config,
                    image_name= docker_repo,
                    train_instance_count=1,
                    train_instance_type=instance_type,
                    hyperparameters =hyperparameters,
                    output_path=s3_output_path,
                    metric_definitions=metric_definitions,
                    #train_use_spot_instances = True
                    train_volume_size=30,
                    code_location=s3_code_path,
                    base_job_name ="ppi-bert-extractor-neg")

In [ ]:
estimator.fit(inputs)